# Feature Engineering

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [4]:

df=pd.read_csv("../data/processed/cleaned_subscriptions_churn.csv")
df.head()

,customer_id,gender,months_subscribed,streaming_quality,subscription_type,payment_mode,monthly_plan_cost,total_revenue,subscription_canceled,app_usage_hours,last30d_usage_hours,customer_rating,promo_email_clicks,device_type,num_profiles,auto_renew,support_tickets_last6m,nps_score,is_active_last30d
0,7590-VHVEG,Female,1,HD,Month-to-month,Electronic check,29.85,29.85,0,4.1,125.9,3,3,Mobile,1,1,0,9,1
1,5575-GNVDE,Male,34,HD,One year,Mailed check,56.95,1889.50,0,5.1,177.9,4,2,Mobile,1,0,0,7,1
2,3668-QPYBK,Male,2,HD,Month-to-month,Mailed check,53.85,108.15,1,2.4,88.5,5,4,Desktop,1,1,0,1,1
3,7795-CFOCW,Male,45,HD,One year,Bank transfer (automatic),42.30,1840.75,0,3.8,124.0,5,5,SmartTV,1,0,0,6,1
4,9237-HQITU,Female,2,4K,Month-to-month,Electronic check,70.70,151.65,1,6.3,182.8,5,2,Mobile,2,1,0,3,1


In [5]:
#MISSSING VALUES Treatment
df.isnull().sum()
# since the cleaned dataset has no missing values or already treated out hence we dont have to work on missing value treatment

customer_id               0
gender                    0
months_subscribed         0
streaming_quality         0
subscription_type         0
payment_mode              0
monthly_plan_cost         0
total_revenue             0
subscription_canceled     0
app_usage_hours           0
last30d_usage_hours       0
customer_rating           0
promo_email_clicks        0
device_type               0
num_profiles              0
auto_renew                0
support_tickets_last6m    0
nps_score                 0
is_active_last30d         0
dtype: int64

In [6]:
# Saperation of Numerical and Categorical.

num_data= df.select_dtypes(include=[np.number])
num_col=list(num_data.columns)

cat_data= df.select_dtypes(include=[object]) #since numpy has removed the np.objects and pandas can inbult detect the categorical/text as object, hence we used "object" instead of "np.object"
cat_col=list(cat_data.columns)

In [10]:
#Outliers treatment
num_col
#Detection - will go one by one on each numerical feature
real_num=['months_subscribed','monthly_plan_cost','total_revenue','app_usage_hours','last30d_usage_hours']
real_num
# we will start Outliers detection by simple z-score or IQR Box plots
# Before going forward we need to check skewness first, since high skewed data sometimes would not get captured by IQR or z-score test.
# by EDA we get to know that total_revenue(highly right) and last30d_usage_hours(mild) are skewed .
# Hence we will do MAD score test for these 2 and do z-score test for rest.



['months_subscribed',
 'monthly_plan_cost',
 'total_revenue',
 'app_usage_hours',
 'last30d_usage_hours']